In [4]:
import os
import pandas as pd

In [7]:
%%bash
git clone https://github.com/Jaehun-Kim22/SCIC_Project.git

assets
README.md


fatal: destination path 'SCIC_Project' already exists and is not an empty directory.


In [8]:
## loads excel
xls = pd.ExcelFile('SCIC_Project/assets/data.xlsx')
training_data = pd.read_excel(xls, '학습데이터')
priority = pd.read_excel(xls, '우선순위')

In [9]:
print(training_data, priority)

       NO    NO2  ... 발화3의 우선순위 최종분류(우선순위 가장 높은것 선택)
0       1   6438  ...       NaN       불만>고객서비스>상담시스템
1       2   6850  ...       NaN       불만>고객서비스>상담시스템
2       3   7294  ...       NaN       불만>고객서비스>상담시스템
3       4   9484  ...       NaN       불만>고객서비스>상담시스템
4       5  10350  ...       NaN       불만>고객서비스>상담시스템
..    ...    ...  ...       ...                  ...
995  9958    103  ...       NaN         칭찬>삼성카드>카드상품
996  9961      2  ...       NaN           칭찬>삼성카드>혜택
997  9962      4  ...       NaN           칭찬>삼성카드>혜택
998  9963      5  ...       NaN           칭찬>삼성카드>혜택
999  9964      7  ...       NaN           칭찬>삼성카드>혜택

[1000 rows x 10 columns]     우선순위           의도명(유형)
0      1      칭찬>고객서비스>상담원
1      2    칭찬>고객서비스>상담시스템
2      3          칭찬>고객서비스
3      4      불만>고객서비스>상담원
4      5    불만>고객서비스>상담시스템
5      6          불만>고객서비스
6      7        칭찬>삼성카드>혜택
7      8    칭찬>삼성카드>할부금융상품
8      9   칭찬>삼성카드>커뮤니티서비스
9     10   칭찬>삼성카드>카드이용/결제
10    11      칭찬>삼성카드>카드상품
11    12      